In [ ]:
dat = read.table(file="cookies.dat", header=TRUE)
head(dat)

In [ ]:
table(dat$location)

In [ ]:
boxplot(chips ~ location, data = dat)

In [ ]:
set.seed(112)
n_sim = 500
alpha_pri = rexp(n_sim, 1/2)
beta_pri = rexp(n_sim, 5.0)

mu_pri = alpha_pri / beta_pri
sig_pri = sqrt(alpha_pri / beta_pri^2)
summary(mu_pri)

In [ ]:
summary(sig_pri)

In [ ]:
lam_pri = rgamma(n_sim, shape = alpha_pri, rate = beta_pri)

In [ ]:
summary(lam_pri)b

In [ ]:
y_pri = rpois(n_sim, lambda = lam_pri)

In [ ]:
hist(y_pri[y_pri < 100], breaks=50, col = 'blue', xlim=c(0, 100))
hist(lam_pri[lam_pri < 100], breaks=50, col = 'green', add=TRUE)
hist(mu_pri[mu_pri < 100], breaks=50, col = 'yellow', add=TRUE)

In [ ]:
source("../shared.R")

In [ ]:
mod_str = "model {
    for (i in 1:length(chips)) {
        chips[i] ~ dpois(lam[location[i]])
    }
    
    for (j in 1:max(location)) {
        lam[j] ~ dgamma(alpha, beta)
    }
    
    # we assign priors on mean and variances rather than alpha and beta direcly    
    mu ~ dgamma(2., 1/5.)
    sig ~ dexp(1.0)    
    alpha = mu^2 / sig^2
    beta = mu / sig^2    
}"

set.seed(113)
data_jags = as.list(dat)

params = c("lam", "mu", "sig", "alpha", "beta")

In [ ]:
mod = jags_model_compile(mod_str, data_jags)

In [ ]:
mod_sim = jags_model_sample(mod_ = mod, var_names_ = params, n_iter_ = 5e3)
mod_diag = jags_model_diag(samples_ = mod_sim$samples)

In [ ]:
#plot(mod_sim$samples)

In [ ]:
dic1 = dic.samples(mod, n.iter = 1e4)
dic1

## Model Checking (via residuals)

There are 2 levels of residuals, one for mean level, the other is for chips levels

In [ ]:
pm_params = colMeans(mod_sim$samples_cmb)
pm_params

In [ ]:
yhat  = rep(pm_params[3:7], each=30)
resid = dat$chips - yhat

In [ ]:
#plot(resid)

Because we expect the resid distributed as Poisson, we expect the var resid will be eq to $\lambda$

If its bigger that $\lambda$ we observe the problem of overdispersion 

and probably would like to change underlying distribution to something different, like Neg-Binomial or something different

In [ ]:
var(resid[yhat < 7])

In [ ]:
plot(jitter(yhat), resid)

In [ ]:
lam_resid  = pm_params[3:7] - pm_params['mu']
plot(lam_resid)

In [ ]:
summary(mod_sim$samples_cmb)

## Posterior predictive simulations

In [ ]:
n = length(mod_sim$samples_cmb[,'alpha'])
lam_pred = rgamma(n, shape = mod_sim$samples_cmb[,'alpha'], rate = mod_sim$samples_cmb[,'beta'])

In [ ]:
mean(lam_pred > 15)

In [ ]:
y_pred = rpois(n, lambda = lam_pred)

In [ ]:
hist(y_pred, freq=FALSE, colr= 'blue')
hist(dat$chips, freq=FALSE, add=TRUE, color='green')